# Nettoyage Base Nationale des Batiments

A partir du module `preprocessing` de `lib`, 
- Encodage des variables
- Sélection des variables pertinentes 
- Sélection des observations dont les identifiants `parcelle_id` sont dans DVF. 

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import sys 
sys.path.append("../")

from lib.preprocessing import bnb, dvf

In [9]:
import gc 
gc.collect()

237

In [10]:
DATA_DIR = "../data/"
ZIP_DIR = f"{DATA_DIR}dvf_cleaned.zip" 

BACKUP_DIR = f"{DATA_DIR}dvf+/"

In [4]:
bnb_df = bnb.load_bnb(DATA_DIR, "base_nat_bat.parquet")

In [11]:
bnb_df.shape 

(27607773, 59)

In [12]:
dvf_args = {
    "geo_area": "Paris", 
    "property_type": "flats", 
}

dvf_df = dvf.concat_datasets_per_year(ZIP_DIR, **dvf_args)

Processing 2018:  17%|█▋        | 1/6 [00:13<01:09, 13.97s/it]

In [ ]:
dvf_df

In [7]:
dvfplus = bnb.create_dvfplus(
    dvf=dvf_df, 
    bnb=bnb_df,
    na_max=.2
) 

Loading DVF dataset...


Processing 2022: 100%|██████████| 6/6 [04:32<00:00, 45.44s/it]


Preprocessing Base Nationale des Batiments dataset...
Merging DVF and Base Nationale des Batiments datasets...


TypeError: merge() got multiple values for argument 'how'

In [ ]:
dvfplus

In [137]:
file_path = f"{BACKUP_DIR}{dvf_args['geo_area']}_{dvf_args['property_type']}.csv"
dvfplus.to_csv(file_path, index=False)